In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
!pip install kaggle
!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d stevezhenghp/airbnb-price-prediction

 80% 25.0M/31.3M [00:00<00:00, 57.4MB/s]
100% 31.3M/31.3M [00:00<00:00, 70.6MB/s]


In [5]:
!unzip airbnb-price-prediction.zip

Archive:  airbnb-price-prediction.zip
  inflating: train.csv               


In [6]:
dataset = pd.read_csv("train.csv")
dataset.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [0]:
dataset = dataset.fillna(0)

In [0]:
dataset[['cleaning_fee']] = dataset[['cleaning_fee']].astype(int)


In [0]:
dataset[['host_identity_verified','instant_bookable']] = (dataset[['host_identity_verified','instant_bookable']] == 't').astype(int) 

In [0]:
X = dataset.iloc[:,[2,3,5,6,8,9,14,17,23,24,27,28]].values
Y = dataset.iloc[:,1].values

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder1 = LabelEncoder()
X[:,0] = labelencoder1.fit_transform(X[:,0])
labelencoder2 = LabelEncoder()
X[:,1] = labelencoder2.fit_transform(X[:,1])
labelencoder3 = LabelEncoder()
X[:,4] = labelencoder3.fit_transform(X[:,4])


In [22]:
X[3]

array([17, 0, 4, 1.0, 0, 1, 1, 0, 0, 0.0, 2.0, 2.0], dtype=object)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [24]:
X[3]

array([ 1.30366109, -0.85067289,  0.39230329, -0.39661178, -1.32947177,
        0.60187992,  0.69980585, -0.59653531, -0.55251017, -1.82470155,
        0.8627925 ,  0.23277841])

In [25]:
X.shape

(74111, 12)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [0]:
#model 1
from keras.models import Sequential
from keras.layers import Dense,Dropout

regressor = Sequential()
regressor.add(Dense(256, kernel_initializer='normal', input_dim = X.shape[1], activation = "relu"))
regressor.add(Dropout(0.2))
              
#hidden layers
regressor.add(Dense(units = 128,activation = "relu"))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 64,activation = "relu"))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1,activation = "linear"))

regressor.compile(optimizer = "adam", loss = "mean_absolute_error",metrics = ["mean_absolute_error"])

regressor.fit(X_train,Y_train,batch_size = 32,epochs = 25,validation_data = (X_test,Y_test))


In [70]:
#model 2
regressor = Sequential()
regressor.add(Dense(512, kernel_initializer='normal', input_dim = X.shape[1], activation = "relu"))
regressor.add(Dropout(0.2))
              
#hidden layers
regressor.add(Dense(units = 256,activation = "relu"))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 128,activation = "relu"))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 64,activation = "relu"))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1,activation = "linear"))

regressor.compile(optimizer = "adam", loss = "mean_absolute_error",metrics = ["mean_absolute_error"])

regressor.fit(X_train,Y_train,batch_size = 32,epochs = 35,validation_data = (X_test,Y_test))

Train on 66699 samples, validate on 7412 samples
Epoch 1/35
66699/66699 [==============================] - 23s 349us/step - loss: 0.6031 - mean_absolute_error: 0.6031 - val_loss: 0.3931 - val_mean_absolute_error: 0.3931
Epoch 2/35
66699/66699 [==============================] - 22s 327us/step - loss: 0.4824 - mean_absolute_error: 0.4824 - val_loss: 0.3778 - val_mean_absolute_error: 0.3778
Epoch 3/35
66699/66699 [==============================] - 34s 516us/step - loss: 0.4537 - mean_absolute_error: 0.4537 - val_loss: 0.3788 - val_mean_absolute_error: 0.3788
Epoch 4/35
66699/66699 [==============================] - 26s 396us/step - loss: 0.4364 - mean_absolute_error: 0.4364 - val_loss: 0.3761 - val_mean_absolute_error: 0.3761
Epoch 5/35
66699/66699 [==============================] - 33s 488us/step - loss: 0.4232 - mean_absolute_error: 0.4232 - val_loss: 0.3766 - val_mean_absolute_error: 0.3766
Epoch 6/35
66699/66699 [==============================] - 27s 401us/step - loss: 0.4146 - mean_a

In [71]:
regressor.predict(X[0:5,:])

array([[5.0107875],
       [5.309187 ],
       [4.881216 ],
       [5.150697 ],
       [4.709445 ]], dtype=float32)

In [72]:
regressor.predict(X_test)

array([[5.1576433],
       [4.945154 ],
       [4.3945646],
       ...,
       [4.855028 ],
       [6.082535 ],
       [4.885177 ]], dtype=float32)

In [0]:
Y_test = Y_test.reshape(7412,1)

In [0]:
ans = (regressor.predict(X_test)-Y_test)

In [78]:
print(ans)

[[ 0.55247313]
 [-0.0654811 ]
 [ 0.31702718]
 ...
 [-0.12170559]
 [-0.22738349]
 [-0.54854487]]


In [79]:
print("Mean absolute error")
print(np.sum(abs(ans),axis = 0)/X_test.shape[0])

Mean absolute error
[0.37402977]
